In [2]:
import json

with open("scenes/scenes.json", "r") as f:
    scenes = json.load(f)   # scenes should be a list of dicts

from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "./qwen_sft_vr"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).cuda()

def generate_output(prompt, max_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    output = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=False,    # or True if you want sampling
        temperature=0.0,
    )

    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return text[len(prompt):].strip()  # optionally strip prompt


FileNotFoundError: [Errno 2] No such file or directory: 'scenes/scenes.json'

In [ ]:
completions = []
correct = 0  # for pass@1 metric

for scene in scenes:
    prompt = scene["prompt"]
    solution = scene["html"]

    prediction = generate_output(prompt)

    completions.append({
        "scene_id": scene["scene_id"],
        "prompt": prompt,
        "solution": solution,
        "completion": prediction
    })

    # crude match check — adjust your comparison logic!
    if prediction.strip() == solution.strip():
        correct += 1

with open("completions.json", "w") as f:
    json.dump(completions, f, indent=2)

In [ ]:
# Computer pass@1.
pass_1 = correct / len(scenes)
print("pass@1:", pass_1)